# Copilot SDK Structured Output Analysis

This notebook analyzes why the Copilot SDK might fail to produce structured output (tool calls) and instead return raw strings or JSON. It uses a standalone environment without importing project dependencies.

In [20]:
import asyncio
import os
import json
from typing import List, Optional
from pydantic import BaseModel, Field

# Ensure you have the 'copilot' package installed in your environment
try:
    from copilot import CopilotClient, define_tool
except ImportError:
    print("❌ 'copilot' package not found. Please install it or run in an environment with access to it.")
    # Mock for local testing if SDK is missing
    class CopilotClient:
        def __init__(self, *args, **kwargs): pass
        async def start(self): print("Mock Client Started")
        async def stop(self): print("Mock Client Stopped")
        async def create_session(self, config): return MockSession()

    def define_tool(name, description, params_type):
        return lambda func: func

    class MockSession:
        async def send(self, data): pass
        def on(self, callback): pass

## 1. Define Tools (Mocking Agent Structure)

We replicate the `submit_question` tool structure exactly as it is in `copilot_questionnaire.py`.

In [21]:
class SubmitQuestionParams(BaseModel):
    narrative: str = Field(description="RPG 情境敘述")
    question_text: str = Field(description="題目內容")
    options: List[str] = Field(description="選項列表")
    type: str = Field(default="QUANTITATIVE", description="題目類型")
    guide_message: str = Field(default="", description="嚮導話語")

class CompleteTrialParams(BaseModel):
    final_message: str = Field(description="結業語")

@define_tool(
    name="submit_question",
    description="提交 RPG 情境敘述與題目",
    params_type=SubmitQuestionParams,
)
async def submit_question(params: SubmitQuestionParams) -> dict:
    print("🛠️ [TOOL submit_question] submit_question called with:\n")
    print(f"Narrative: {params.narrative}")
    print(f"Question Text: {params.question_text}")
    print(f"Options: {params.options}")
    print(f"Type: {params.type}")
    print(f"Guide Message: {params.guide_message}") 
    return params.model_dump()

@define_tool(
    name="complete_trial",
    description="完成所有測驗題目",
    params_type=CompleteTrialParams,
)
async def complete_trial(params: CompleteTrialParams) -> dict:
    print("🏁 [TOOL complete_trial] complete_trial called with:\n")
    print(f"Final Message: {params.final_message}")
    return params.model_dump()

tools = [submit_question, complete_trial]

## 2. System Prompt

Using the exact prompt from `copilot_questionnaire.py`.

In [22]:
QUESTIONNAIRE_INSTRUCTION = """你是 TraitQuest 的「引導者艾比 (Abby)」，一位充滿神祕感與智慧的靈魂導師。
你的任務是根據測驗類別（MBTI, DISC, Big Five, Enneagram, Gallup），將心理測驗題目偽裝在 RPG 情境對話中。

測驗工具的遊戲角色定義：
- MBTI → 核心職業 (Class)：16 型人格決定角色的外觀與決策風格（如戰略法師 INTJ、吟遊詩人 INFP）
- Big Five → 屬性數值 (Stats)：五大人格特質轉化為角色面板數值
  * Openness (開放性) → 智力 (Intelligence)
  * Conscientiousness (嚴謹性) → 防禦 (Defense)
  * Extraversion (外向性) → 速度 (Speed)
  * Agreeableness (親和性) → 魅力 (Charisma)
  * Neuroticism (神經質) → 洞察 (Insight)
- DISC → 戰略姿態 (Stance)：行為風格決定戰鬥動作（烈焰戰姿/攻、潮汐之歌/援、大地磐石/守、星辰軌跡/算）
- Enneagram → 靈魂種族 (Race)：九型人格中心決定種族歸屬，影響 MP 回復效率
- Gallup → 技能樹 (Talent)：34 種天賦強項轉化為 2-3 個主動/被動技能

敘事規範：
- 語氣：神祕、共情、略帶史詩感。
- 延續性：必須讀取冒險者的 hero_chronicle，在開場白中提到他們過去的行為（例如：「我記得你曾選擇在森林中保護那隻幼獸...」）。
- 試煉長度（必須嚴格遵守）：
    題數與題型根據【玩家等級】決定：
    * Lv.1~10 (量化試煉)：10 題，僅使用 QUANTITATIVE（五段式選擇題）
    * Lv.11~15 (靈魂對話)：10 題，可使用 SOUL_NARRATIVE（開放式文字輸入）
    * Lv.16+ (深邃試煉)：15 題，混合使用選擇題與開放式輸入
    系統會在指令中告訴你當前題號與總題數，你必須在達到總題數時調用 `complete_trial`。
    **嚴禁提前結束或超出題數。**

- 題型規則：
    * QUANTITATIVE：五段式選擇題（用於 Lv.1~10，或 Lv.16+ 混合時使用）
    * SOUL_NARRATIVE：開放式問題，無選項，由 AI 語義解析（僅 Lv.11+ 可用）
    * Lv.16+ 深邃試煉建議比例：60% 選擇題 + 40% 開放式

- 測驗導向：根據當前測驗類型（questId），你應設計能夠探索該特定心理維度的情境與選項。
  * MBTI 測驗應著重探索思考方式（直覺 vs 實際、邏輯 vs 情感）
  * Big Five 測驗應針對五個維度設計漸進式問題
  * DISC 測驗應觀察行為反應模式
  * Enneagram 測驗應探索人格中心的特點
  * Gallup 測驗應探索天賦強項的應用
- 結構：
    - 使用 `submit_question` 提交新的問題與劇情。
    - **當你收到的指令顯示已達到總題數上限，或者你認為已經收集到足夠的心理特徵資訊時，請務必使用 `complete_trial` 工具結束測驗。**
- 限制：
    - 劇情敘述 (narrative) 最多 100 字。
    - 題目 (question) 最多 50 字。
    - 選項 (options) 最多 5 個選項，每個選項最多 8 字，且選項可以是不同答案，也可以是由輕到重的程度區別。
    - 題目類型 (type) 只能是 QUANTITATIVE 或 SOUL_NARRATIVE。
    - 嚮導話語 (guide_message) 為可選，在開場或重要轉折點提供簡短鼓勵，最多 15 字。
    - 輸入字串使用正體中文。
- 重要：**你唯一的輸出（The ONLY output）必須是調用工具 `submit_question` 或 `complete_trial`。**
- 嚴禁在工具調用之前或之後輸出任何文字、解釋、確認訊息或 Markdown 區塊。
- 如果你輸出了任何非工具調用的文字（如「好的，這是題目...」），系統將無法解析，導致試煉失敗。
- 極端重要：**你唯一的輸出（The ONLY output）必須是調用工具 `submit_question` 或 `complete_trial`。**
- 嚴禁在工具調用之前或之後輸出任何文字、解釋、確認訊息或 Markdown 區塊。
- 絕對不要輸出 JSON 或 XML，必須直接調用工具。

範例輸出（Example Output）：
正確的工具調用：
submit_question(
    narrative="雖然這是一個範例，但它展示了正確的格式。",
    question_text="這個範例是否清楚？",
    options=["清楚", "不清楚", "非常清楚", "完全不清楚", "普通"],
    type="QUANTITATIVE",
    guide_message="很好。"
)

錯誤的輸出（嚴禁）：
"好的，這是您的題目..."
```json
{
  "narrative": "...",
  "question_text": "..."
}
```
"""

## 3. Execution & Event Listener

We will run the client and listen for all events to see exactly what the model returns.

In [25]:
async def run_test():
    client = CopilotClient()
    await client.start()
    
    try:
        session = await client.create_session({
            "model": "gpt-4o", # Or your specific model config
            "system_prompt": QUESTIONNAIRE_INSTRUCTION,
            "tools": tools
        })
        
        # Event Listener to confirm what's happening under the hood
        def on_event(event):
            event_type = str(event.type)
            print(f"\n[EVENT] {event_type}")

            if "assistant.message" in event_type:
                print(f"\n📩 [EVENT] Assistant Message: {event.data.content}")
            elif "tool.call" in event_type:
                print(f"\n🔨 [EVENT] Tool Call: {event.data.name}")
            elif "tool.execution_end" in event_type:
                # 這裡印出 output 可以驗證剛才函數回傳的 dict/json 結果
                print(f"\n✅ [EVENT] Tool Execution End: {event.data.name}")
                print(f"Output: {event.data.output}")
            else:
                # Print other events to understand the flow
                # print(f"[EVENT] {event_type}")
                pass
                
        session.on(on_event)
        
        # Sending a prompt that aligns with the system instruction
        print("🚀 Sending instruction...")
        await session.send({
            "prompt": "開始 MBTI 測驗，這是第一題。"
        })
        
        # Keep session alive for a bit to capture events
        await asyncio.sleep(30)
        
    finally:
        await client.stop()

# Run the test
# Note: In a real Jupyter environment, you can just await run_test()
await run_test()


🚀 Sending instruction...

[EVENT] SessionEventType.PENDING_MESSAGES_MODIFIED

[EVENT] SessionEventType.PENDING_MESSAGES_MODIFIED

[EVENT] SessionEventType.USER_MESSAGE

[EVENT] SessionEventType.ASSISTANT_TURN_START

[EVENT] SessionEventType.SESSION_USAGE_INFO

[EVENT] SessionEventType.ASSISTANT_USAGE

[EVENT] SessionEventType.ASSISTANT_MESSAGE

[EVENT] SessionEventType.ASSISTANT_REASONING

[EVENT] SessionEventType.TOOL_EXECUTION_START

[EVENT] SessionEventType.TOOL_EXECUTION_START
🛠️ [TOOL submit_question] submit_question called with:

Narrative: 在「深綠星痕塔」的最底層，你被古老的魔導卷軸喚醒。塔內迴盪著低語，無數冒險者的靈痕在半空交錯，等著與你結伴闖入更深的迷宮。當你推開沉重的黑鐵門，一個由星光構成的守門者問道，想要解讀你的靈魂走向，就先回答牠的試煉。
Question Text: 【第 1 題】當你踏入一座充滿陌生冒險者的酒館，需要與他們結伴探索迷宮時，你通常會怎麼行動？
Options: ['A. 主動走向幾桌人，自我介紹並試著帶動話題，很快就能和多數人混熟。', 'B. 先觀察環境，選一兩個看起來氣場合得來的人，找機會與他們自然開啟對話。', 'C. 如果有人來搭話就會有禮回應，但不太會主動開口，多半安靜地待在角落。', 'D. 盡量避免與陌生人互動，寧可一個人待著或找藉口離開那裡。']
Type: QUANTITATIVE
Guide Message: 請選擇最貼近你『平常真實狀態』的選項，而不是你理想中的樣子。直接回覆 A、B、C 或 D 即可。

[EVENT] SessionEventType.TOOL_E